In [1]:
import tensorflow as tf
from tensorflow.keras import Model
from tensorflow.keras.layers import Conv2D,BatchNormalization,MaxPooling2D,Input,GlobalAveragePooling2D,Dense
from tensorflow.keras.activations import relu
from tensorflow.keras.preprocessing.image import ImageDataGenerator

/opt/conda/lib/python3.10/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.24.3
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


## Residual block

In [2]:
class Block(Model):
    def __init__(self,input_channels,output_channels,use_conv=False,identity_strides=1):
        super().__init__()
        self.conv1=Conv2D(input_channels,kernel_size=1,strides=identity_strides)
        self.conv2=Conv2D(input_channels,kernel_size=3,strides=1,padding='same')
        self.conv3=Conv2D(output_channels,kernel_size=1,strides=1,padding='same')
        self.conv_identity=None
        if use_conv:
            self.conv_identity=Conv2D(output_channels,kernel_size=1,strides=identity_strides,padding='same')
            
        self.bn1=BatchNormalization()
        self.bn2=BatchNormalization()
        self.bn3=BatchNormalization()
        self.bn_identity =BatchNormalization()
        
    def call(self,X):
        #Traditional residuals
        Y=self.conv1(X)
        Y = self.bn1(Y)
        Y= relu(Y)
        
        Y=self.conv2(Y)
        Y = self.bn2(Y)
        Y= relu(Y)
        
        Y=self.conv3(Y)
        Y = self.bn3(Y)
        Y= relu(Y)
        
        if self.conv_identity is not None:
            X=self.conv_identity(X)
            X=self.bn_identity(X)
        
        Y+=X
        Y=relu(Y)
        return Y

In [3]:
base_layer = tf.keras.models.Sequential([
    Conv2D(input_shape=(224,224,3),filters=64,kernel_size=7,strides=2,padding='same'),
    BatchNormalization(),
    tf.keras.layers.Activation("relu"),
    MaxPooling2D(pool_size=3, strides=2, padding='same')
    
])
base_layer.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 112, 112, 64)      9472      
                                                                 
 batch_normalization (Batch  (None, 112, 112, 64)      256       
 Normalization)                                                  
                                                                 
 activation (Activation)     (None, 112, 112, 64)      0         
                                                                 
 max_pooling2d (MaxPooling2  (None, 56, 56, 64)        0         
 D)                                                              
                                                                 
Total params: 9728 (38.00 KB)
Trainable params: 9600 (37.50 KB)
Non-trainable params: 128 (512.00 Byte)
_________________________________________________________________


In [4]:
train_directory='/100-bird-species/train' # train path
val_directory='/100-bird-species/valid' # validation path
test_directory='/100-bird-species/test' # test path

In [5]:
train_datagen=ImageDataGenerator(
 rescale=1/255.0, 
 rotation_range=10, 
 zoom_range=0.05, 
 width_shift_range=0.05, 
 height_shift_range=0.05, 
 shear_range=0.05,
 horizontal_flip=True,
 fill_mode='nearest')

batch_size =256 

train_generator = train_datagen.flow_from_directory(
    directory=train_directory,
    target_size=(224,224),
    color_mode='rgb',
    batch_size=batch_size,
    class_mode='categorical', 
    shuffle=True, 
    seed=42
)
val_datagen=ImageDataGenerator(rescale=1/255.0)
valid_generator = val_datagen.flow_from_directory(
    directory=val_directory,
    target_size=(224, 224),
    color_mode="rgb",
    batch_size=batch_size,
    class_mode="categorical",
    shuffle=True,
    seed=42
)
test_datagen = ImageDataGenerator(rescale=1/255.0) 
test_generator = test_datagen.flow_from_directory(
  directory=test_directory,
  target_size=(224,224),
    color_mode='rgb',
    batch_size=batch_size,
    class_mode=None,
    shuffle=False,
    seed=42
)

Found 84635 images belonging to 525 classes.
Found 2625 images belonging to 525 classes.
Found 2625 images belonging to 525 classes.


In [6]:
strategy = tf.distribute.MirroredStrategy()

## Resnet 50

In [7]:
from tensorflow.keras.layers import Input, Conv2D, BatchNormalization, MaxPooling2D, GlobalAveragePooling2D, Dense
from tensorflow.keras.models import Model

def ResNet50(input_shape=(224, 224, 3), input_tensor=None, classes=1000):
    if input_tensor is None:
        X_input = Input(shape=input_shape)
    else:
        if not backend.is_keras_tensor(input_tensor):
            X_input = Input(tensor=input_tensor, shape=input_shape)
        else:
            X_input = input_tensor
            
    X = Conv2D(filters=64, kernel_size=7, strides=2, padding='same')(X_input)
    X = BatchNormalization()(X)
    X = relu(X)
    X = MaxPooling2D(pool_size=3, strides=2, padding='same')(X)

    # Stage 1
    X = Block(64, 256, use_conv=True, identity_strides=1)(X)
    X = Block(256, 256, use_conv=False)(X)
    X = Block(256, 256, use_conv=False)(X)

    # Stage 2
    X = Block(256, 512, use_conv=True, identity_strides=2)(X)
    for _ in range(3):
        X = Block(512, 512, use_conv=False)(X)

    # Stage 3
    X = Block(512, 1024, use_conv=True, identity_strides=2)(X)
    for _ in range(5):
        X = Block(1024, 1024, use_conv=False)(X)

    # Stage 4
    X = Block(1024, 2048, use_conv=True, identity_strides=2)(X)
    for _ in range(2):
        X = Block(2048, 2048, use_conv=False)(X)

    X = GlobalAveragePooling2D()(X)
    X = Dense(classes, activation='softmax')(X)

    model = Model(inputs=X_input, outputs=X, name='ResNet50')
    return model


In [8]:
with strategy.scope():
    model =ResNet50(input_shape=(224,224,3),classes=525)
    model.summary()

Model: "ResNet50"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 224, 224, 3)]     0         
                                                                 
 conv2d_1 (Conv2D)           (None, 112, 112, 64)      9472      
                                                                 
 batch_normalization_1 (Bat  (None, 112, 112, 64)      256       
 chNormalization)                                                
                                                                 
 tf.nn.relu (TFOpLambda)     (None, 112, 112, 64)      0         
                                                                 
 max_pooling2d_1 (MaxPoolin  (None, 56, 56, 64)        0         
 g2D)                                                            
                                                                 
 block (Block)               (None, 56, 56, 256)       769

In [9]:
with strategy.scope():
    lr=0.0001
    epochs=30
    model.compile(loss='categorical_crossentropy',optimizer=tf.keras.optimizers.legacy.Adam(learning_rate=lr,decay=lr/(epochs/0.5)),metrics=['acc'])

In [10]:
with strategy.scope():
    model.fit(train_generator,validation_data=valid_generator,
          steps_per_epoch=train_generator.n//train_generator.batch_size,
          validation_steps=valid_generator.n//valid_generator.batch_size,
          epochs=epochs
         )

Epoch 1/30
330/330 [==============================] - 1770s 5s/step - loss: 5.1779 - acc: 0.0758 - val_loss: 8.5781 - val_acc: 0.0023
Epoch 2/30
330/330 [==============================] - 1237s 4s/step - loss: 3.2317 - acc: 0.3079 - val_loss: 2.9216 - val_acc: 0.3543
Epoch 3/30
330/330 [==============================] - 1224s 4s/step - loss: 2.3004 - acc: 0.4813 - val_loss: 2.2815 - val_acc: 0.4703
Epoch 4/30
330/330 [==============================] - 1217s 4s/step - loss: 1.7664 - acc: 0.5912 - val_loss: 1.9761 - val_acc: 0.5281
Epoch 5/30
330/330 [==============================] - 1203s 4s/step - loss: 1.4247 - acc: 0.6655 - val_loss: 2.2048 - val_acc: 0.5234
Epoch 6/30
330/330 [==============================] - 1221s 4s/step - loss: 1.1807 - acc: 0.7182 - val_loss: 1.7942 - val_acc: 0.5969
Epoch 7/30
330/330 [==============================] - 1208s 4s/step - loss: 0.9947 - acc: 0.7596 - val_loss: 1.5978 - val_acc: 0.6199
Epoch 8/30
330/330 [==============================] - 1217s 4s

In [11]:
# model.save("path/to/location")

/opt/conda/lib/python3.10/site-packages/keras/src/engine/training.py:3000: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(
